In [1]:
import pandas as pd
from datetime import datetime as dt

import integrate as intr


In [2]:

miv_df = intr.process_miv_data()
fb_data = intr.process_foot_bike_data()

/Users/seb/Projects/repos/group-1/src/integrate.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  miv_df_cols_dropped['Weekday_en'] = days.map(lambda x: weekday_names[x])


In [ ]:
test_df = miv_df


In [ ]:
test_df.dtypes
date_object = pd.to_datetime(test_df['Date'])


In [ ]:
test_df['Weekday_Name']

In [3]:
acc_df = intr.process_accident_data(True)

u_string RoadTrafficAccidentLocations.json
Filepath: datasets/RoadTrafficAccidentLocations.json


/Users/seb/Projects/repos/group-1/src/integrate.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_acc_df.rename(columns={


In [ ]:
acc_df.head()
acc_df['AccidentWeekDay_en'].unique()
#acc_df.dtypes



In [4]:
print("Accident Columns:")
print(acc_df.dtypes)
print()
print("MIV Columns:")
print(miv_df.dtypes)
print()
print("FB Cols:")
print(fb_data.dtypes)

Accident Columns:
AccidentUID                      object
AccidentHour                     object
AccidentYear                     object
AccidentWeekDay_en               object
AccidentType                     object
AccidentSeverityCategory         object
AccidentInvolvingPedestrian      object
AccidentInvolvingBicycle         object
AccidentInvolvingMotorcycle      object
RoadType                         object
RoadType_en                      object
EKoord                           object
NKoord                           object
AccidentMonth                    object
geometry                       geometry
dtype: object

MIV Columns:
MSID                   object
ZSID                   object
Achse                  object
EKoord                float64
NKoord                float64
Richtung               object
AnzFahrzeuge          float64
AnzFahrzeugeStatus     object
Date                   object
Hrs                    object
Weekday_en             object
dtype: object

FB Cols:


In [ ]:
acc_df['ID'] = acc_df.index +1
acc_df[('ID')]

In [10]:
print("MIV unqiue:", miv_df['EKoord'])
print("Acc unique:", acc_df['RoadType'].unique)
print("FB unique: ", fb_data['DATE'])


MIV unqiue: 0           2683009.89
1           2683009.89
2           2683009.89
3           2683009.89
4           2683009.89
               ...    
16699185    2682704.50
16699186    2682704.50
16699187    2682704.50
16699188    2682704.50
16699189    2682704.50
Name: EKoord, Length: 16699190, dtype: float64
Acc unique: <bound method Series.unique of 0        rt433
1        rt433
2        rt439
3        rt433
4        rt433
         ...  
55821    rt432
55822    rt433
55823    rt433
55824    rt433
55825    rt432
Name: RoadType, Length: 55826, dtype: object>
FB unique:  0          2012-01-01
1          2012-01-01
2          2012-01-01
3          2012-01-01
4          2012-01-01
              ...    
3011488    2019-07-13
3011489    2019-07-13
3011490    2019-07-13
3011491    2019-07-13
3011492    2019-07-13
Name: DATE, Length: 3011493, dtype: object


In [5]:
acc_df.head()

,AccidentUID,AccidentHour,AccidentYear,AccidentWeekDay_en,AccidentType,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_en,EKoord,NKoord,AccidentMonth,geometry
0,A2D2677533867004E0430A865E337004,00,2011,Saturday,at0,as4,false,false,false,rt433,Minor road,2684605,1245194,1,POINT Z (8.55841 47.35217 0.00000)
1,9FD6441F802C20A6E0430A865E3320A6,01,2011,Saturday,at0,as3,false,true,false,rt433,Minor road,2682382,1246980,1,POINT Z (8.52932 47.36851 0.00000)
2,9FDA0DC4856A6094E0430A865E336094,02,2011,Saturday,at0,as4,false,false,false,rt439,Other,2682791,1247749,1,POINT Z (8.53488 47.37538 0.00000)
3,A3B66E42396E6000E0430A865E336000,02,2011,Saturday,at5,as3,false,false,false,rt433,Minor road,2681199,1247102,1,POINT Z (8.51368 47.36976 0.00000)
4,9FDA0DBE8CCE9096E0430A865E339096,03,2011,Saturday,at0,as4,false,false,false,rt433,Minor road,2682479,1250690,1,POINT Z (8.53129 47.40186 0.00000)


In [ ]:
from sqlalchemy import create_engine
from geoalchemy2 import Geometry, WKTElement
import geopandas as gpd
from shapely import wkt

db_url = f'postgresql://seb:@localhost:5432/test-db23'
engine =  create_engine(db_url)

#miv_df.to_sql('table_name', engine, if_exists='replace', index=False)
#fb_data.to_sql('footbike', engine, if_exists='replace', index=False)

geometry_column = 'geometry'


acc_df['geometry'] = acc_df['geometry'].apply(lambda geom: WKTElement(geom.wkt, srid=4326))

acc_df.to_sql('accidents', engine, if_exists='replace', index=False, dtype={'geometry': Geometry('POINT', srid=4326)})



In [ ]:
engine.dispose()